In [1]:
from roaringbitmap import RoaringBitmap
from itertools import product
import matplotlib.pyplot as plt
import time
import csv
import heapq as hq
import networkx as nx
import numpy as np
from tensorflow.keras.models import load_model

2025-05-31 10:51:01.255006: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-31 10:51:01.397136: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-31 10:51:01.439386: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-31 10:51:01.714630: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-31 10:51:03.671534: W tensorflow/compiler/tf2

In [2]:
tam = 30
final_model_dnn_path = f"modelos/dnn_model_{tam}.keras"
final_model_value_path = f"modelos/dnn_value_model_{tam}.keras"
model_branching = load_model(final_model_dnn_path)
model_bounding = load_model(final_model_value_path)

I0000 00:00:1748699465.133853    6070 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748699465.401218    6070 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748699465.401313    6070 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748699465.404164    6070 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1748699465.404259    6070 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Algoritmo de branch and bound com paralelismo de bits

In [3]:
def smallest_last_degree(G):

	residual_degree = {}
	Q = []
	for v in G.nodes:
		residual_degree[v] = len( G.adj[v] )
		Q.append( (residual_degree[v], v) )

	hq.heapify(Q)

	S = []
	while len(Q) > 0:
		(degree, v) = hq.heappop(Q)
		if v in S:
			continue
		S.append(v)
		for u in G.adj[v]:
			if u not in S:
				residual_degree[u] -= 1
				hq.heappush(Q, (residual_degree[u], u))
	S.reverse()
	return S

In [4]:
def max_kpartite(adj, V, l):
    index = 1
    R = V.copy()
    sizeR = len(R)

    while sizeR > 0 and index <= l:
        S = R.copy()
        while len(S) > 0:
            v = S.min()
            S -= adj[v]
            R.remove(v)
            S.remove(v)
            sizeR -= 1
        index += 1
    return R

In [5]:
def bitcliquePartite(G, V, clique_atual, best_clique, number_of_nodes):
    number_of_nodes[0] += 1
    sizeV = len(V)

    if sizeV == 0:
        if len(clique_atual) > len(best_clique):
            best_clique[:] = clique_atual[:]
    else:
        l = len(best_clique) - len(clique_atual)
        R = max_kpartite(G, V, l)

        order = list(R)

        m = len(order)
        for i in range(m-1, -1, -1):
            v = order[i]
            V2 = V.copy()
            V2 &= G[v]
            V.remove(v)
            clique_atual.append(v)
            bitcliquePartite(G, V2, clique_atual, best_clique, number_of_nodes)
            clique_atual.pop()

In [6]:
def algoBitCliqueMaxPartite(G):
	start_time = time.time()

	S = smallest_last_degree(G)
	rotulo = {}
	for i in range( len(S) ):
		rotulo[ S[i] ] = i
	G = nx.relabel_nodes(G, rotulo)

	adj = []
	n = len(G.nodes)
	for i in range( n ):
		adj.append(  RoaringBitmap( list( G.adj[i])  ) )

	clique_atual = []
	best_clique = []
	V = RoaringBitmap( [ i for i in range(n) ])
	number_of_nodes = [ 0 ]


	bitcliquePartite( adj ,  V , clique_atual, best_clique, number_of_nodes )

	end_time = time.time()
	elapsed_time = end_time - start_time
	return best_clique, number_of_nodes, elapsed_time

Algoritmo de branch and bound com DLTS

In [7]:
def build_state_representation(Q, K, G_dict_of_lists, tam):
    vertex_list = list(K)
    len_k = len(vertex_list)

    vertex_to_idx = {vertex: i for i, vertex in enumerate(vertex_list)}

    adj_matrix = np.zeros((tam, tam), dtype=np.float32)
    clique_mask = np.zeros(tam, dtype=np.float32)

    if not Q:
        return adj_matrix, clique_mask

    v_q = Q[-1]

    G_adj_sets_for_K = {node_k: set(G_dict_of_lists.get(node_k, [])) for node_k in vertex_list}

    neighbors_of_vq_in_G_set = set(G_dict_of_lists.get(v_q, []))
    
    neighbors_of_vq_in_K = neighbors_of_vq_in_G_set & K

    if v_q in vertex_to_idx:
        idx_vq = vertex_to_idx[v_q]
        v_q_is_in_NvqK = (v_q in neighbors_of_vq_in_K)

        for k_idx in range(len_k):
            vk = vertex_list[k_idx]
            vk_is_in_NvqK = (vk in neighbors_of_vq_in_K)

            if v_q_is_in_NvqK or vk_is_in_NvqK:
                if vk in neighbors_of_vq_in_G_set:
                    adj_matrix[idx_vq, k_idx] = 1.0
            
            if vk_is_in_NvqK or v_q_is_in_NvqK:
                if v_q in G_adj_sets_for_K[vk]:
                    adj_matrix[k_idx, idx_vq] = 1.0

    Q_set = set(Q)
    for i in range(len_k):
        if vertex_list[i] in Q_set:
            clique_mask[i] = 1.0
            
    return adj_matrix, clique_mask

In [8]:
def build_inputs_for_model(adj_matrix, clique_mask):
    tam = adj_matrix.shape[0]

    if tam == 0:
        inputs = []
    else:
        inputs = np.vsplit(adj_matrix, tam)

    inputs.append(clique_mask[np.newaxis, :])
    
    return inputs

In [9]:
def dnn_bounding(Q, K, G, tam):
    adj_matrix, clique_mask = build_state_representation(Q, K, G, tam)
    inputs = build_inputs_for_model(adj_matrix, clique_mask)
    pred = model_bounding.predict(inputs, verbose=0)
    valor = pred[0, 0]  # extrai o escalar do array
    return int(round(valor))

def get_branching_dnn_predictions(Q, K, G_permuted, tam):
    if not K: # Se K estiver vazio, não há o que pontuar.
        return np.array([]) # Retorna um array vazio ou None, dependendo de como o chamador lida.

    adj_matrix, clique_mask = build_state_representation(Q, K, G_permuted, tam)
    inputs = build_inputs_for_model(adj_matrix, clique_mask)
    pred_raw = model_branching.predict(inputs, verbose=0) # Saída é (1, tam)
    return pred_raw[0]

def should_expand(Q_new, K_new, C, G, tam):
    est_total = len(Q_new) + dnn_bounding(Q_new, K_new, G, tam)
    return est_total > len(C)

def neighbors(G, v):
    return set(G[v])

In [10]:
def pre_process_instance(G_original):
    original_node_labels = sorted(G_original.keys(), key=lambda v_orig: -len(G_original.get(v_orig, [])), reverse=False)

    p_vertex_position = {v_orig: i_permuted for i_permuted, v_orig in enumerate(original_node_labels)}
    p_vertex_at_position = {i_permuted: v_orig for i_permuted, v_orig in enumerate(original_node_labels)}

    permuted_G = {}
    num_nodes = len(original_node_labels)
    for i_permuted in range(num_nodes):
        v_orig = p_vertex_at_position[i_permuted]
        original_neighbors = G_original.get(v_orig, [])
        permuted_G[i_permuted] = {p_vertex_position[u_orig] for u_orig in original_neighbors}
    
    C_permuted = []
    K_permuted_initial = set(permuted_G.keys()) # Nós de 0 a num_nodes-1
    S = [([], K_permuted_initial)] # Q inicial vazio, K inicial com todos os nós permutados

    return C_permuted, S, permuted_G, p_vertex_at_position

In [11]:
def post_process_instance(C_permuted, p_vertex_at_position):
    return [p_vertex_at_position[i_permuted] for i_permuted in C_permuted]

In [12]:
def MCBB_DLTS(G_original, 
              tam,  
              k_iteracoes_refresh_dnn_branching=2, 
              k_profundidade_para_dnn_bound=3):
    
    start_time = time.time()
    nodes_expandidos_count = 0

    C_permuted, S, G_permuted, map_vertex_at_position = pre_process_instance(G_original)

    scores_branching_atuais = None
    iteracoes_desde_ultimo_refresh_branching = 0

    while S:
        Q_raiz_perm, K_raiz_perm = S.pop()
        nodes_expandidos_count += 1

        Q_caminho_perm = list(Q_raiz_perm)
        K_caminho_perm = set(K_raiz_perm)

        iteracoes_desde_ultimo_refresh_branching = 0
        scores_branching_atuais = None 

        while K_caminho_perm:
            extensao_estimada_clique = 0
            
            if k_profundidade_para_dnn_bound <= 0:
                profundidade_check_dnn_bound = 0 
            else:
                profundidade_check_dnn_bound = len(Q_caminho_perm) % k_profundidade_para_dnn_bound
            
            if profundidade_check_dnn_bound == 0:
                extensao_estimada_clique = dnn_bounding(
                                                Q_caminho_perm, 
                                                K_caminho_perm, 
                                                G_permuted, tam)
            else:
                extensao_estimada_clique = len(K_caminho_perm) 

            if not (len(C_permuted) < len(Q_caminho_perm) + extensao_estimada_clique):
                break

            if iteracoes_desde_ultimo_refresh_branching % k_iteracoes_refresh_dnn_branching == 0 or \
                scores_branching_atuais is None:
                scores_branching_atuais = get_branching_dnn_predictions(
                                                Q_caminho_perm, K_caminho_perm, 
                                                G_permuted, tam)
            
            iteracoes_desde_ultimo_refresh_branching += 1
            
            v_perm = None
            if scores_branching_atuais is not None and len(scores_branching_atuais) == tam:
                scores_candidatos_K = {idx_node: scores_branching_atuais[idx_node] 
                                    for idx_node in K_caminho_perm}
                if scores_candidatos_K:
                    v_perm = max(scores_candidatos_K, key=scores_candidatos_K.get)
            
            if v_perm is None:
                break 
            
            K_caminho_perm_sem_v = set(K_caminho_perm)
            K_caminho_perm_sem_v.remove(v_perm)

            if K_caminho_perm_sem_v:
                if should_expand(Q_caminho_perm, K_caminho_perm_sem_v, C_permuted, G_permuted, tam):
                    S.append((list(Q_caminho_perm), K_caminho_perm_sem_v))
            
            Q_caminho_perm.append(v_perm)
            K_caminho_perm = K_caminho_perm_sem_v.intersection(neighbors(G_permuted, v_perm))
        
        if len(C_permuted) < len(Q_caminho_perm):
            C_permuted = list(Q_caminho_perm)

    elapsed_time = time.time() - start_time
    C_final_original = post_process_instance(C_permuted, map_vertex_at_position)
    
    return C_final_original, nodes_expandidos_count, elapsed_time

Comparação entre os Algoritmos de branch and bound com paralelismo de bits e com DLTS

In [13]:
data = [ "nodes", "densidade", "DLTS Clique (tempo)" , "algoBitCliqueMaxPartite (tempo)", "DLTS Clique (nodes)", "algoBitCliqueMaxPartite (nodes)", "Dif"]
with open('exato.csv', 'w', encoding='UTF8', newline='') as f:
  writer = csv.writer(f)
  writer.writerow(data)

valores1 = [10, 15, 20, 30]
valores2 = [0.25, 0.5, 0.75, 0.9]

combinacoes = list(product(valores1, valores2))
for c in combinacoes:
  for i in range(10):
    nodes = c[0]
    densidade = c[1]
    G = nx.gnp_random_graph(nodes, densidade)
    G_dict = nx.to_dict_of_lists(G)
    best_clique1, number_of_nodes1, elapsed_time1 = MCBB_DLTS(G_dict, tam)
    best_clique2, number_of_nodes2, elapsed_time2 = algoBitCliqueMaxPartite(G)
    # comparar se o tamanho do clique é o mesmo
    tam_dlts = len(best_clique1)
    tam_exato = len(best_clique2)
    dif = (tam_exato - tam_dlts) / tam_exato

    data = [ nodes, densidade, elapsed_time1, elapsed_time2, number_of_nodes1, number_of_nodes2[0], dif ]
    with open('exato.csv', 'a', encoding='UTF8', newline='') as f:
      writer = csv.writer(f)
      writer.writerow(data)

/home/igor/anaconda3/envs/tcc/lib/python3.9/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['input_0', 'input_1', 'input_2', 'input_3', 'input_4', 'input_5', 'input_6', 'input_7', 'input_8', 'input_9', 'input_10', 'input_11', 'input_12', 'input_13', 'input_14', 'input_15', 'input_16', 'input_17', 'input_18', 'input_19', 'input_20', 'input_21', 'input_22', 'input_23', 'input_24', 'input_25', 'input_26', 'input_27', 'input_28', 'input_29', 'input_30']. Received: the structure of inputs=('*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*')
  warnings.warn(
I0000 00:00:1748699476.612169   13306 service.cc:146] XLA service 0x7fbb1c00a5d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748699476.612240   13306 service.cc:154]   StreamExecutor device (0): NVIDIA

In [22]:
dados_agrupados = []
try:
    with open('exato.csv', 'r', encoding='UTF8') as f:
        reader = csv.reader(f)
        header_exato = next(reader) # Pula o cabeçalho do exato.csv

        current_group = []
        i = 0
        for row_idx, row in enumerate(reader):
            converted_row = []
            # Colunas esperadas no exato.csv e seus índices:
            # 0: nodes (ignorado aqui, pois será pego de 'combinacoes' depois)
            # 1: densidade (ignorado aqui)
            # 2: DLTS Clique (tempo) -> float
            # 3: algoBitCliqueMaxPartite (tempo) -> float
            # 4: DLTS Clique (nodes) -> int
            # 5: algoBitCliqueMaxPartite (nodes) -> int
            # 6: Dif -> float
            try:
                # Adiciona apenas as colunas de dados numéricos que serão promediadas
                converted_row.append(float(row[2])) # DLTS Tempo
                converted_row.append(float(row[3])) # BitClique Tempo
                converted_row.append(int(row[4]))   # DLTS Nodes
                converted_row.append(int(row[5]))   # BitClique Nodes
                converted_row.append(float(row[6])) # Dif
            except ValueError as e:
                print(f"Erro ao converter valor na linha {row_idx + 2} do exato.csv: {e}. Linha: {row}")
                continue # Pula esta linha se houver erro de conversão

            current_group.append(converted_row)
            i += 1
            
            if i % 10 == 0:
                if current_group: 
                    dados_agrupados.append(current_group)
                current_group = []
        
        if current_group: # Adiciona o último grupo se ele não completou 10 linhas
            dados_agrupados.append(current_group)

except FileNotFoundError:
    print("Erro: O arquivo 'exato.csv' não foi encontrado.")
    exit()
except Exception as e:
    print(f"Ocorreu um erro ao ler 'exato.csv': {e}")
    exit()

# Parte 2: Cálculo das Médias
medias_agrupadas = []
if not dados_agrupados:
    print("Nenhum dado foi agrupado de 'exato.csv'. Verifique o arquivo e a lógica de agrupamento.")
else:
    for grupo_idx, grupo in enumerate(dados_agrupados):
        if not grupo:
            print(f"Aviso: Grupo {grupo_idx} está vazio. Pulando.")
            continue

        media_grupo = []
        num_colunas_para_media = 5 
        for col_idx in range(num_colunas_para_media):
            coluna_valores = []
            for linha in grupo:
                if col_idx < len(linha):
                    coluna_valores.append(linha[col_idx])
                else:
                    print(f"Aviso: Linha no grupo {grupo_idx} não tem coluna {col_idx}. Linha: {linha}")

            if coluna_valores:
                media_calculada = sum(coluna_valores) / len(coluna_valores)
                media_grupo.append(media_calculada)
            else:
                print(f"Aviso: Coluna {col_idx} no grupo {grupo_idx} não tem valores. Usando 0.0 para média.")
                media_grupo.append(0.0) 
        medias_agrupadas.append(media_grupo)

# Parte 3: Escrita de 'medias.csv' com formatação
if 'combinacoes' not in locals() or not isinstance(combinacoes, list) or not combinacoes:
    print("Erro: A variável 'combinacoes' não está definida, não é uma lista ou está vazia.")
    print("Defina 'combinacoes' como uma lista de tuplas (nodes, densidade) antes de executar esta parte.")
    # Exemplo para demonstração (REMOVA OU SUBSTITUA PELA SUA LÓGICA REAL)
    # Se você tem 16 grupos em medias_agrupadas, combinacoes deve ter 16 pares.
    # Ex: combinacoes = [(10,0.25)]*len(medias_agrupadas) # Placeholder
    exit()

if len(combinacoes) != len(medias_agrupadas):
    print(f"Erro: O número de combinações ({len(combinacoes)}) não corresponde ao número de grupos de médias ({len(medias_agrupadas)}).")
    exit()

try:
    with open('medias.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)
        header_medias = ["nodes", "densidade", "DLTS Clique (tempo medio)", 
                         "algoBitCliqueMaxPartite (tempo medio)", "DLTS Clique (nodes medio)", 
                         "algoBitCliqueMaxPartite (nodes medio)", "Dif medio"]
        writer.writerow(header_medias)
        
        for i, (nodes_val, densidade_val) in enumerate(combinacoes):
            if i < len(medias_agrupadas) and len(medias_agrupadas[i]) == 5:
                data_row = [
                    nodes_val,
                    densidade_val,
                    "{:.2e}".format(medias_agrupadas[i][0]),  # DLTS Clique (tempo medio)
                    "{:.2e}".format(medias_agrupadas[i][1]),  # algoBitCliqueMaxPartite (tempo medio)
                    "{:.2e}".format(medias_agrupadas[i][2]),  # DLTS Clique (nodes medio)
                    "{:.2e}".format(medias_agrupadas[i][3]),  # algoBitCliqueMaxPartite (nodes medio)
                    "{:.2f}".format(medias_agrupadas[i][4])   # Dif medio - FORMATADO COMO X.XX
                ]
                writer.writerow(data_row)
            else:
                print(f"Aviso: Dados de média ausentes ou incompletos para combinação {i} ({nodes_val}, {densidade_val}). Pulando escrita.")

    print("Arquivo medias.csv salvo com sucesso e com a formatação desejada!")

except Exception as e:
    print(f"Ocorreu um erro ao escrever 'medias.csv': {e}")

Arquivo medias.csv salvo com sucesso e com a formatação desejada!


In [19]:
import io
import pandas as pd

csv_data_medias = """nodes,densidade,DLTS Clique (tempo medio),algoBitCliqueMaxPartite (tempo medio),DLTS Clique (nodes medio),algoBitCliqueMaxPartite (nodes medio),Dif medio
10,0.25,7.18,0.0,13.5,1.38e+01,0.00e+00
10,0.5,7.77,0.0,17.0,1.71e+01,0.00e+00
10,0.75,5.83,0.0,13.2,2.53e+01,2.00e-02
10,0.9,5.25,0.0,11.3,3.58e+01,3.00e-02
15,0.25,10.99,0.0,25.7,1.96e+01,0.00e+00
15,0.5,9.05,0.0,20.9,2.95e+01,3.00e-02
15,0.75,10.16,0.0,19.6,4.40e+01,3.00e-02
15,0.9,7.38,0.0,12.8,6.68e+01,3.00e-02
20,0.25,17.04,0.0,39.3,2.61e+01,0.00e+00
20,0.5,16.8,0.0,36.0,3.47e+01,0.00e+00
20,0.75,9.06,0.0,16.2,6.50e+01,5.00e-02
20,0.9,9.83,0.0,16.1,9.67e+01,6.00e-02
30,0.25,39.22,0.0,94.0,4.00e+01,0.00e+00
30,0.5,26.17,0.0,50.4,5.74e+01,8.00e-02
30,0.75,12.09,0.0,21.2,1.04e+02,1.00e-01
30,0.9,16.13,0.0,25.5,1.74e+02,4.00e-02
"""

# Usa io.StringIO para tratar a string como um arquivo
data_file_like_object = io.StringIO(csv_data_medias)

# Lê os dados CSV para um DataFrame do pandas
df_medias = pd.read_csv(data_file_like_object)

# Imprime o DataFrame. Ele será formatado como uma tabela.
print(df_medias.to_string())

    nodes  densidade  DLTS Clique (tempo medio)  algoBitCliqueMaxPartite (tempo medio)  DLTS Clique (nodes medio)  algoBitCliqueMaxPartite (nodes medio)  Dif medio
0      10       0.25                       7.18                                    0.0                       13.5                                   13.8       0.00
1      10       0.50                       7.77                                    0.0                       17.0                                   17.1       0.00
2      10       0.75                       5.83                                    0.0                       13.2                                   25.3       0.02
3      10       0.90                       5.25                                    0.0                       11.3                                   35.8       0.03
4      15       0.25                      10.99                                    0.0                       25.7                                   19.6       0.00
5      15       